<a href="https://colab.research.google.com/github/riyagpt0251/CartPoleBalancing/blob/main/CartPoleBalancing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Step 1: Install Required Libraries
!pip install gym numpy



In [3]:
# Step 2: Import Libraries
import gym
import numpy as np



In [4]:
# Step 3: Set Up the Environment
env = gym.make('CartPole-v1')



/usr/local/lib/python3.11/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.11/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [5]:
# Step 4: Define Q-Learning Parameters
# Discretize the state space
num_buckets = (1, 1, 6, 12)  # (cart position, cart velocity, pole angle, pole angular velocity)
num_actions = env.action_space.n  # Number of possible actions (left or right)



In [6]:
# Q-table
Q = np.zeros(num_buckets + (num_actions,))

# Hyperparameters
alpha = 0.1  # Learning rate
gamma = 0.99  # Discount factor
epsilon = 1.0  # Exploration rate
epsilon_decay = 0.995
epsilon_min = 0.01



In [7]:
# Step 5: Discretize the State Space
def discretize_state(state):
    discretized_state = []
    for i in range(len(state)):
        scale = (state[i] + abs(env.observation_space.low[i])) / (env.observation_space.high[i] - env.observation_space.low[i])
        discretized_state.append(int(round((num_buckets[i] - 1) * scale)))
        discretized_state[i] = min(num_buckets[i] - 1, max(0, discretized_state[i]))
    return tuple(discretized_state)


In [8]:

# Step 6: Implement Q-Learning
def q_learning(episodes):
    global epsilon
    for episode in range(episodes):
        state = discretize_state(env.reset())
        done = False
        total_reward = 0

        while not done:
            # Exploration vs exploitation
            if np.random.rand() < epsilon:
                action = env.action_space.sample()  # Explore
            else:
                action = np.argmax(Q[state])  # Exploit

            # Take action
            next_state, reward, done, _ = env.step(action)
            next_state = discretize_state(next_state)

            # Update Q-value
            Q[state + (action,)] += alpha * (reward + gamma * np.max(Q[next_state]) - Q[state + (action,)])

            state = next_state
            total_reward += reward

        # Decay epsilon
        epsilon = max(epsilon_min, epsilon * epsilon_decay)

        if (episode + 1) % 100 == 0:
            print(f"Episode: {episode + 1}, Total Reward: {total_reward}, Epsilon: {epsilon}")



In [9]:
# Step 7: Train the Agent
q_learning(episodes=1000)



<ipython-input-7-8b96d6f42942>:5: RuntimeWarning: overflow encountered in scalar subtract
  scale = (state[i] + abs(env.observation_space.low[i])) / (env.observation_space.high[i] - env.observation_space.low[i])
/usr/local/lib/python3.11/dist-packages/gym/utils/passive_env_checker.py:241: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Episode: 100, Total Reward: 34.0, Epsilon: 0.6057704364907278
Episode: 200, Total Reward: 12.0, Epsilon: 0.3669578217261671
Episode: 300, Total Reward: 11.0, Epsilon: 0.22229219984074702
Episode: 400, Total Reward: 43.0, Epsilon: 0.1346580429260134
Episode: 500, Total Reward: 18.0, Epsilon: 0.08157186144027828
Episode: 600, Total Reward: 42.0, Epsilon: 0.0494138221100385
Episode: 700, Total Reward: 25.0, Epsilon: 0.029933432588273214
Episode: 800, Total Reward: 35.0, Epsilon: 0.018132788524664028
Episode: 900, Total Reward: 39.0, Epsilon: 0.01098430721937979
Episode: 1000, Total Reward: 24.0, Epsilon: 0.01


In [10]:
# Step 8: Evaluate the Agent
def evaluate_agent(episodes=100):
    total_rewards = []
    for episode in range(episodes):
        state = discretize_state(env.reset())
        done = False
        total_reward = 0

        while not done:
            action = np.argmax(Q[state])  # Always exploit
            state, reward, done, _ = env.step(action)
            state = discretize_state(state)
            total_reward += reward

        total_rewards.append(total_reward)
        print(f"Evaluation Episode: {episode + 1}, Total Reward: {total_reward}")

    avg_reward = np.mean(total_rewards)
    print(f"Average Reward over {episodes} episodes: {avg_reward}")

# Evaluate the trained agent
evaluate_agent(episodes=100)

Evaluation Episode: 1, Total Reward: 33.0
Evaluation Episode: 2, Total Reward: 26.0
Evaluation Episode: 3, Total Reward: 38.0
Evaluation Episode: 4, Total Reward: 44.0
Evaluation Episode: 5, Total Reward: 36.0
Evaluation Episode: 6, Total Reward: 39.0
Evaluation Episode: 7, Total Reward: 35.0
Evaluation Episode: 8, Total Reward: 34.0
Evaluation Episode: 9, Total Reward: 25.0
Evaluation Episode: 10, Total Reward: 45.0
Evaluation Episode: 11, Total Reward: 39.0
Evaluation Episode: 12, Total Reward: 25.0
Evaluation Episode: 13, Total Reward: 26.0
Evaluation Episode: 14, Total Reward: 37.0
Evaluation Episode: 15, Total Reward: 35.0
Evaluation Episode: 16, Total Reward: 41.0
Evaluation Episode: 17, Total Reward: 36.0
Evaluation Episode: 18, Total Reward: 41.0
Evaluation Episode: 19, Total Reward: 35.0
Evaluation Episode: 20, Total Reward: 45.0
Evaluation Episode: 21, Total Reward: 47.0
Evaluation Episode: 22, Total Reward: 37.0
Evaluation Episode: 23, Total Reward: 39.0
Evaluation Episode: 

<ipython-input-7-8b96d6f42942>:5: RuntimeWarning: overflow encountered in scalar subtract
  scale = (state[i] + abs(env.observation_space.low[i])) / (env.observation_space.high[i] - env.observation_space.low[i])


Evaluation Episode: 37, Total Reward: 31.0
Evaluation Episode: 38, Total Reward: 34.0
Evaluation Episode: 39, Total Reward: 44.0
Evaluation Episode: 40, Total Reward: 35.0
Evaluation Episode: 41, Total Reward: 30.0
Evaluation Episode: 42, Total Reward: 39.0
Evaluation Episode: 43, Total Reward: 41.0
Evaluation Episode: 44, Total Reward: 41.0
Evaluation Episode: 45, Total Reward: 42.0
Evaluation Episode: 46, Total Reward: 39.0
Evaluation Episode: 47, Total Reward: 35.0
Evaluation Episode: 48, Total Reward: 25.0
Evaluation Episode: 49, Total Reward: 32.0
Evaluation Episode: 50, Total Reward: 31.0
Evaluation Episode: 51, Total Reward: 31.0
Evaluation Episode: 52, Total Reward: 34.0
Evaluation Episode: 53, Total Reward: 40.0
Evaluation Episode: 54, Total Reward: 36.0
Evaluation Episode: 55, Total Reward: 33.0
Evaluation Episode: 56, Total Reward: 31.0
Evaluation Episode: 57, Total Reward: 38.0
Evaluation Episode: 58, Total Reward: 26.0
Evaluation Episode: 59, Total Reward: 44.0
Evaluation 